# README    
1. generate a Keras Layer :AttLayer  
2. Use csv module to import data


# Result  
1)使用Dense层
acc在.73
roc在0.79
说明数据集没有问题

# Read Data

In [57]:
datafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/pairs.csv'
import csv
import numpy as np

In [58]:
with open(datafile) as csvfile:
    spamreader = csv.reader(csvfile)
    for  row in spamreader:
        csvkeys = row
        break
print csvkeys

['bin', 'enhancer_chrom', 'enhancer_distance_to_promoter', 'enhancer_end', 'enhancer_name', 'enhancer_start', 'label', 'promoter_chrom', 'promoter_end', 'promoter_name', 'promoter_start', 'window_end', 'window_start', 'window_chrom', 'window_name', 'interactions_in_window', 'active_promoters_in_window']


In [59]:

poscsv = {}
negcsv = {}
with open(datafile) as csvfile:
    reader = csv.DictReader(csvfile)
    for line in reader:
        if line['bin']  not in poscsv.keys():
            poscsv[line['bin']] =[]
        if line ['bin']  not in negcsv.keys():
            negcsv[line['bin']] =[]

        record=[ line['enhancer_chrom']+' '+
                line[ 'enhancer_start']+' '+
                line['enhancer_end']+' '+
                line['promoter_chrom']+' '+
                line['promoter_start']+' '+
                line['promoter_end']+' '+
                line['enhancer_name']+'-'+line['promoter_name']
                  ]
        if line['label']==str(1):
            poscsv[line['bin']].append(record)
        else:
            negcsv[line['bin']].append(record)

    for key in poscsv.keys():
        length = len(poscsv[key])
        index = range(len(negcsv[key]))
        np.random.shuffle(index)
        negcsv[key] = np.array(negcsv[key])
        negcsv[key] = negcsv[key][index[:length]]
        #print len(negcsv[key])

In [60]:
posdata =[]
negdata = []
for key in poscsv.keys():
    posdata = np.append(posdata,poscsv[key])
    negdata = np.append(negdata,negcsv[key])
assert  len(posdata) == len(negdata)
print  len(posdata),len(negdata)

2113 2113


In [61]:
from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')

In [62]:
print range(4,9),range(0,6)

[4, 5, 6, 7, 8] [0, 1, 2, 3, 4, 5]


In [63]:
#获得ｋｍｅｒ的函数
embed_dict ={}
def seq_to_embed_seq(seq):
    kmer=6
    embed_seq = []
    if len (seq)< kmer:
        return embed_seq
    for i in range(kmer,len(seq)):
        word  = seq[(i-kmer):i] 
        #if  word not in embed_dict.keys():
        #    embed_dict[ word  ] =str( len(embed_dict.keys()) ) #给标号
        #embed_seq.append(  str( embed_dict[ word  ]  )  )
        embed_seq.append(  word )
    return embed_seq
        
        


In [64]:
range(8)[3-2:3+2]

[1, 2, 3, 4]

In [65]:
sentence_len = 806
half_len = sentence_len/2

PESeq=dict()
PESeq['index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-ESeq']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-ESeq']=[]
PESeq['label'] =[]

#正样本
for index in range(len(posdata)):
    line = posdata[index]
    #print line
    line = line.split()
    ehr_seq = genome[line[0]][int(line[1]): int(line[2]) ].upper()
    pro_seq = genome [line[3]][int(line[4]) :int(line[5])].upper()
    index = line[6]
    
    if len(pro_seq)<6:
        print line
        continue
    if len(ehr_seq)<6:
        print line
        continue
    
    middle = ( int(line[1])+int(line[2]))//2
    ehr_seq = genome[line[0]][  (middle-half_len): (middle+half_len)   ].upper()
    pro_seq = genome [line[3]][  (middle-half_len): (middle+half_len)  ].upper()
    
    PESeq['index'].append(index )
    PESeq['Pro-Seq'].append(pro_seq)
    #print len(pro_seq)-len(seq_to_embed_seq(pro_seq))
    #print seq_to_embed_seq(pro_seq)
    
    PESeq['Pro-ESeq'].append( seq_to_embed_seq(pro_seq))
    PESeq['Ehr-Seq'].append(ehr_seq)
    #print seq_to_embed_seq(ehr_seq)
    #break
    PESeq['Ehr-ESeq'].append(seq_to_embed_seq(ehr_seq))
    PESeq['label'].append(1)
print 'pos over'
print len(PESeq['label'])  

#负样本
  
for  index in range(len(negdata)):
    #break
    line = negdata[index]
    line = line.split()
    ehr_seq = genome[line[0]][int(line[1]): int(line[2]) ].upper()
    pro_seq = genome [line[3]][int(line[4]) :int(line[5])].upper()
    index = line[6]
    
    #负样本比较多 可以严格限制 不可以 设置为1000 只剩168
    if len(pro_seq)<6:
        print line
        continue
    if len(ehr_seq)<6:
        print line
        continue
     
    middle = ( int(line[1])+int(line[2]))//2
    ehr_seq = genome[line[0]][  (middle-half_len): (middle+half_len)   ].upper()
    pro_seq = genome [line[3]][  (middle-half_len): (middle+half_len)  ].upper()
    
    PESeq['index'].append(index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-ESeq'].append( seq_to_embed_seq(pro_seq))
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-ESeq'].append(seq_to_embed_seq(ehr_seq))
    PESeq['label'].append(0)
print 'neg over'

print len(PESeq['label'])

['chr17', '41080595', '41080600', 'chr17', '40728821', '40732200', 'GM12878|chr17:41080595-41080600-GM12878|chr17:40728821-40732200']
['chr3', '144015595', '144015600', 'chr3', '143689613', '143692895', 'GM12878|chr3:144015595-144015600-GM12878|chr3:143689613-143692895']
['chr6', '27857195', '27857200', 'chr6', '27803860', '27807953', 'GM12878|chr6:27857195-27857200-GM12878|chr6:27803860-27807953']
pos over
2110
['chr16', '66958800', '66958805', 'chr16', '68055893', '68057989', 'GM12878|chr16:66958800-66958805-GM12878|chr16:68055893-68057989']
['chr6', '2856795', '2856800', 'chr6', '3156975', '3158233', 'GM12878|chr6:2856795-2856800-GM12878|chr6:3156975-3158233']
['chr6', '31793795', '31793800', 'chr6', '31938311', '31941111', 'GM12878|chr6:31793795-31793800-GM12878|chr6:31938311-31941111']
['chr19', '17829600', '17829605', 'chr19', '18043141', '18044747', 'GM12878|chr19:17829600-17829605-GM12878|chr19:18043141-18044747']
['chr1', '160769395', '160769400', 'chr1', '161014145', '1610172

In [66]:
print '-'*50
for item in PESeq.keys():
    print item, np.shape(PESeq[item])
print 'pos data' , sum(int( 1 ) if item ==1 else int(0) for item in PESeq['label'])
print 'neg data',sum(int( 1) if item ==0 else int(0) for item in PESeq['label'])



--------------------------------------------------
index (4218,)
Ehr-Seq (4218,)
label (4218,)
Ehr-ESeq (4218, 800)
Pro-Seq (4218,)
Pro-ESeq (4218, 800)
pos data 2110
neg data 2108


# Word2vec Embeding

In [6]:
from gensim.models import Word2Vec,word2vec
import numpy as np

In [29]:
Pro_Sentense = PESeq['Pro-ESeq']
with open('./Gen_data/14_Pro_sentense.txt','w') as f:
    for row in range(np.shape( Pro_Sentense)[0]):
        Sentense = Pro_Sentense[row]
        f.write( ' '.join(Sentense)+'\n')
        

In [30]:
Ehr_Sentense = PESeq['Ehr-ESeq']
with open('./Gen_data/14_Ehr_sentense.txt','w') as f:
    for row in range(np.shape( Ehr_Sentense)[0]):
        Sentense = Ehr_Sentense[row]
        f.write( ' '.join(Sentense)+'\n')

In [2]:
Pro_sentences = word2vec.LineSentence('./Gen_data/14_Pro_sentense.txt')
model = Word2Vec(Pro_sentences, size=100, window=5, min_count=5, workers=4)
print(model)
#print(model.wv['AAAAAA'])
pro_word_vectors = model.wv

Word2Vec(vocab=4096, size=100, alpha=0.025)


In [3]:
Ehr_sentences = word2vec.LineSentence('./Gen_data/14_Ehr_sentense.txt')
model = Word2Vec(Pro_sentences, size=100, window=5, min_count=5, workers=4)
print(model)
#print(model.wv['AAAAAA'])
ehr_word_vectors = model.wv

Word2Vec(vocab=4096, size=100, alpha=0.025)


In [7]:
print  len(ehr_word_vectors.index2word)
for item in ehr_word_vectors.index2word[:4]:
    print item
print  np.shape(ehr_word_vectors['AAAAAA'])
ehr_word_vectors.word_vec

4096
TTTTTT
AAAAAA
ATTTTT
AAAAAT
(100,)


<bound method KeyedVectors.word_vec of <gensim.models.keyedvectors.KeyedVectors object at 0x7fb6bb1a90d0>>

In [8]:
ehr_word_vectors.index2word[965]

u'TTGAAC'

In [9]:
ehr_word_vectors.index2word.index('TAACTA')


2531

In [10]:
pro_embedding_matrix = np.zeros((4096,100))  #4096个字,每个字100维
for item in range(4096):
    index = pro_word_vectors.index2word[item]
    pro_embedding_matrix[item]= pro_word_vectors[index]

In [11]:
ehr_embedding_matrix = np.zeros((4096,100))  #4096个字,每个字100维
for item in range(4096):
    index = ehr_word_vectors.index2word[item]
    ehr_embedding_matrix[item]= ehr_word_vectors[index]

In [12]:
print np.shape(pro_embedding_matrix)
pro_embedding_matrix[2000]

(4096, 100)


array([-0.66526341, -2.18758082, -3.17889857, -0.3516255 ,  1.4194845 ,
        1.59716058, -2.64628553, -2.03841424,  0.19542363, -3.02977276,
        1.95476043,  0.48411638,  0.17369674,  0.90408862, -1.35204351,
       -1.50673091,  1.84095097,  0.80530483, -0.37319571, -0.34351718,
       -0.30834603,  1.33482087, -1.05478156, -1.2152648 , -0.33839104,
        0.10570761,  0.37969404,  1.27467644,  1.61143684, -0.877904  ,
        0.55074215, -3.17754579, -1.57869995, -0.79958737,  0.06922833,
        0.97938573,  0.33274606, -1.85906291, -4.0350771 ,  1.40179992,
       -0.55582768,  2.50550127,  1.86563015,  1.04470849, -3.43924427,
       -0.02449347,  0.83693945,  1.26272774,  1.99617875, -1.88724959,
       -0.06803232,  0.91416514,  1.01185358,  0.8829487 ,  3.3631084 ,
        1.85665107,  0.92707521, -1.04846418, -2.78248119,  2.17803454,
        1.62269282,  2.54663444,  1.02478051,  0.24032617, -1.94887912,
       -0.07327405, -0.25843757,  1.85820806, -0.88526863,  1.26

import pickle as pkl
output = open('./Gen_data/14_embedding_matrix.pkl','w')
pkl.dump(pro_embedding_matrix,output)
pkl.dump(ehr_embedding_matrix,output)
output.close()

input_file = open('./Gen_data/14_embedding_matrix.pkl','r')
pro_embedding_matrix = pkl.load(input_file)
ehr_embedding_matrix = pkl.load(input_file)
input_file.close()

# 准备输入

pro_input=np.zeros((len(PESeq['label']),800),dtype = int )
ehr_input=np.zeros((len(PESeq['label']),800),dtype=int)

import pandas as pd

pro_input=np.zeros((len(PESeq['label']),800),dtype = str)

pro_input=[]
for item in range(len(PESeq['label'])):
    pro_input.append(PESeq['Pro-ESeq'][item])
    #print PESeq['Pro-ESeq'][item]
    #print  pro_input
    #break

fun_pro = lambda word : pro_word_vectors.index2word.index(word)
pro_input_1 = pd.DataFrame(pro_input)


#print pro_input_1[0]

pro_input_2 =pro_input_1.applymap(fun_pro)

#print pro_input_2
print pro_input_2.as_matrix()[0]


ehr_input = []
for item in range(len(PESeq['label'])):
    ehr_input.append(PESeq['Ehr-ESeq'][item])
fun_ehr = lambda word : ehr_word_vectors.index2word.index(word)
ehr_input_1 = pd.DataFrame(ehr_input)
pro_input_1.applymap(fun_ehr)

In [107]:
pro_input=[]
for item in range(len(PESeq['label'])):
    pro_origin_sentense = PESeq['Pro-ESeq'][item]
    pro_index_sentense = []
    for word in  pro_origin_sentense:
        
        pro_index_sentense.append( pro_word_vectors.index2word.index( word))
    pro_input.append(pro_index_sentense)
    #print pro_index_sentense
print 'pro_fin!'
ehr_input=[]
func = lambda word: ehr_word_vectors.index2word.index(word)
for item in range(len(PESeq['label'])):
    ehr_origin_sentense = PESeq['Ehr-ESeq'][item]
    ehr_index_sentense = []
    for word in  ehr_origin_sentense:
        ehr_index_sentense.append( ehr_word_vectors.index2word.index( word))
    ehr_input.append(pro_index_sentense)
    #print pro_index_sentense
print 'ehr_fin!'

pro_fin!
ehr_fin!


In [108]:
pro_input = np.array(pro_input)
ehr_input = np.array(ehr_input)

In [118]:
print np.shape(pro_input)
print np.shape(pro_input[0])
print np.shape(ehr_input)
#print np,shape(ehr_input[0])

(4218, 800)
(800,)
(4218, 800)


In [ ]:
Y_org = PESeq['label']

import pickle as pkl
output = open('./Gen_data/14_Network_input.pkl','w')
pkl.dump(pro_input,output)
pkl.dump(ehr_input,output)
pkl.dump(PESeq['label'],output)
output.close()


In [21]:
import pickle as pkl
input_file = open('./Gen_data/14_Network_input.pkl','r')
pro_input = pkl.load(input_file)
ehr_input = pkl.load(input_file)
Y_org=pkl.load(input_file)
input_file.close()

# 切分训练集

In [14]:
from numpy.random import shuffle

In [15]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [19]:
Y = to_categorical(Y)

In [22]:
SPLIT_point = int(0.85*len(Y_org))
seq_index = range(len(Y_org))
shuffle(seq_index)

In [23]:
x_train ,y_train =pro_input[:SPLIT_point], Y[:SPLIT_point]
x_valid ,y_valid = pro_input[SPLIT_point:], Y[SPLIT_point:]

x_train_ehr ,x_valid_ehr =ehr_input[:SPLIT_point], ehr_input[SPLIT_point:]

In [24]:
print np.shape(x_train[0]),y_train[0]

(800,) [ 0.  1.]


print len(pro_input[1]),pro_input[1]

ehr_input=[]
for item in range(len(PESeq['label'])):
    pro_origin_sentense = PESeq['Pro-ESeq'][item]
    pro_index_sentense = []
    for word in  pro_origin_sentense:
        
        pro_index_sentense.append( pro_word_vectors.index2word.index( word))
    pro_input.append(pro_index_sentense)

# Network

In [25]:
from keras.layers import Embedding,InputLayer
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten,Merge

In [26]:
from keras import backend as K

In [41]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations
# Attention GRU network
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [27]:
# temp
import numpy as np
pro_input=np.zeros((800,4096),dtype = int )
ehr_input=np.zeros((800,4096),dtype=int)

In [28]:
np.shape(pro_input[0])

(4096,)

In [31]:
pro_input = np.array(pro_input)

In [32]:
np.shape(pro_input)

(800, 4096)

In [33]:
EMBEDDING_DIM =100
MAX_SEQUENCE_LENGTH = 800  
nb_words =4096   #字典的len(keys())

In [41]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedded_sequences = embedding_layer(sequence_input)

In [26]:
ehr_embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
ehr_sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
ehr_embedded_sequences = ehr_embedding_layer(ehr_sequence_input)

In [45]:
ehr_embedded_sequences.output

AttributeError: 'TensorVariable' object has no attribute 'output'

In [36]:
all_embedded_sequences = Merge([embedded_sequences,ehr_embedded_sequences], mode='concat',concat_axis=0)

AttributeError: 'TensorVariable' object has no attribute 'get_output_shape_at'

In [42]:
l_lstm =Bidirectional(LSTM(2,return_sequences=False))(embedded_sequences)


In [43]:
preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_lstm)

In [106]:
model  = Model (sequence_input,preds)

In [107]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 800)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 800, 100)      409600      input_1[0][0]                    
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 4)             1648        embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             10          bidirectional_1[0][0]            
Total params: 411,258
Trainable params: 411,258
Non-

In [94]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=100, batch_size=50)

ValueError: The model expects 2 input arrays, but only received one array. Found: array with shape (3585,)

# other network

In [35]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
ehr_embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [42]:
left = Sequential()
left.add( InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,),input_dtype='int32'))
left.add(embedding_layer)

right = Sequential()
right.add( InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,),input_dtype='int32'))
right.add(ehr_embedding_layer)

model = Sequential()
model.add(Merge([left, right], mode='concat'))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(AttLayer())
model.add(Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005)))
model.compile(loss='mse', optimizer='rmsprop',metrics=['acc'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 800)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 800, 100)      409600                                       
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 800)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 800, 100)      409600                                       
___________________________________________________________________________________________

In [ ]:
model.fit([x_train, x_train_ehr], y_train, batch_size=128, nb_epoch=20, validation_data=([x_valid, x_valid_ehr], y_valid))

Train on 3585 samples, validate on 633 samples
Epoch 1/20
3328/3585 [==========================>...] - ETA: 43s - loss: 0.5719 - acc: 0.5691

In [102]:
print np.shape(x_train_ehr),np.shape(x_train),np.shape(y_train)

(0,) (3585,) (3585, 2)


In [ ]:
pro = model.predict_on_batch([x_valid,x_valid_ehr])

y_pred = [ 1 if item1>item0 else 0 for item0,item1 in pro]

from sklearn import metrics

acc = metrics.accuracy_score(y_valid[:,1],y_pred)
print acc

auc = metrics.roc_auc_score(y_valid[:,1],pro[:,1])
print auc

f1_score = metrics.f1_score(y_valid[:,1],y_pred)
print f1_score



model.save_weights('./Gen_data/14_Att-BLSTM_model.h5')